In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.metrics import (make_scorer, accuracy_score, matthews_corrcoef, confusion_matrix,roc_auc_score,
                             roc_curve, auc)
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import warnings
warnings.filterwarnings('ignore')

def calDM(y_pro): # y_pro 为预测概率-all
    mean = y_pro.mean(axis=1)
    std = y_pro.std(axis=1)
    DM = np.array([])
    for i in range(len(y_pro)):
        normal = norm(mean.iloc[i],std.iloc[i])
        if mean.iloc[i] >= 0.5:
            DM = np.concatenate([DM,np.array([normal.cdf(0.5)])])
        else:
            DM = np.concatenate([DM,np.array([1-normal.cdf(0.5)])])
    return DM # 化合物的d(std-pro)值-all

def show_metrics(y_true, y_pred): # y_true, y_pred 为真实值、预测值
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    se = tp/(tp+fn)
    sp = tn/(tn+fp)
    Qp = tp/(tp+fp)
    Qn = tn/(tn+fn)
    Q   = accuracy_score(y_true, y_pred)
    MCC = matthews_corrcoef(y_true, y_pred)
    # precision  = precision_score(y_true, y_pred)
    # recall     = recall_score(y_true, y_pred)
    return [se,sp,Qp,Qn,Q,MCC]

def evaluation(y_true,y_pred,TrOrTe):
    """TrOrTe:Dataframe,
       里面仅含有一列值（tr或te）,据此提取拆分出y_true,y_pre里面的训练集，测试集
       返回值：Dataframe，其中Index为["Train","Test"]"""
    tr_y_pred = y_pred[TrOrTe=="tr"] # tr_y_pred 为训练集预测值
    te_y_pred = y_pred[TrOrTe=="te"] # te_y_pred 为测试集预测值
    tr_y_true = y_true[TrOrTe=="tr"] # tr_y_true 为训练集真实值
    te_y_true = y_true[TrOrTe=="te"] # te_y_true 为测试集真实值
    metrics_tr = show_metrics(tr_y_true, tr_y_pred)
    metrics_te = show_metrics(te_y_true, te_y_pred)
    d = pd.DataFrame([metrics_tr,metrics_te],columns=["SE","SP","Q+","Q-","Q","MCC"],index=["Train","Test"])
    return d.round(5)

In [ ]:
prodata = pd.read_csv(r"Q:\ALK_1_Wild\8.ALK_300nM_DM\all_16_random_proba.csv")
y_pro = prodata.loc[:,'Model 1C':'Model 8D']

predata = pd.read_csv(r"Q:\ALK_1_Wild\8.ALK_300nM_DM\all_16_random_pred.csv")
y_pred = predata.loc[:,'Model 1C':'Model 8D']
y_true = predata.loc[:,"Activity"]
TrOrTe = predata.loc[:,"trtre-1"]

In [ ]:
#训练集+测试集ROC
# label = [
#     'Model 1A','Model 1B','Model 2A','Model 2B',
#     'Model 3A','Model 3B','Model 4A','Model 4B',
#     'Model 5A','Model 5B','Model 6A','Model 6B',
#     'Model 7A','Model 7B','Model 8A','Model 8B']
label = [
    'Model 1C','Model 1D','Model 2C','Model 2D',
    'Model 3C','Model 3D','Model 4C','Model 4D',
    'Model 5C','Model 5D','Model 6C','Model 6D',
    'Model 7C','Model 7D','Model 8C','Model 8D']


plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['font.size'] = 12
fig = plt.figure(facecolor="white",figsize=(4*3,32))#背景改为白色，创建空画布

tr_y_true = y_true[TrOrTe=="tr"]
te_y_true = y_true[TrOrTe=="te"]
j = 0
for i in range(16):
    tr_y_pro = y_pro[TrOrTe=="tr"]
    tr_y_proi=tr_y_pro.iloc[:,i]
    te_y_pro = y_pro[TrOrTe=="te"] 
    te_y_proi=te_y_pro.iloc[:,i]
    
    tr_auc_value = roc_auc_score(tr_y_true, tr_y_proi)
    te_auc_value = roc_auc_score(te_y_true, te_y_proi)
    tr_fpr,tr_tpr,tr_threshold = roc_curve(tr_y_true, tr_y_proi, pos_label=1,drop_intermediate=False)
    te_fpr,te_tpr,te_threshold = roc_curve(te_y_true, te_y_proi, pos_label=1,drop_intermediate=False)   
    
    ax  = fig.add_subplot(10,4,i+1)#为画布添加绘图区域
    ax.plot(tr_fpr,tr_tpr,color='blue',linewidth=1,label=f"Training set\n(AUC={round(tr_auc_value,3)})")
    ax.plot(te_fpr,te_tpr,color='red',linestyle='--',linewidth=1,label=f"Test set\n(AUC={round(te_auc_value,3)})")    
    ax.plot([0, 1], [0, 1], 'k:', linewidth=1, linestyle='--')#对角线
    ax.set_xlim([-0.05, 1.05])  
    ax.set_ylim([-0.05, 1.05])
    ax.set_yticks(np.arange(0,1.1,0.5))
    ax.set_xticks(np.arange(0,1.1,0.5))
    if i >= 28:
        ax.set_xlabel("False positive rate")
    if i%4 == 0:
        ax.set_ylabel("True positive rate")
    ax.legend(fontsize=11)
    #ax.text(-0.25,1.06,f"({chr(i+97)})",fontdict=dict(fontsize=16))
    ax.set_title(label[i])
plt.tight_layout()
plt.savefig(r"Q:\ALK_1_Wild\8.ALK_300nM_DM\all_16_random_ROC.tif", dpi=600,bbox_inches="tight")
plt.show()

In [ ]:
#遍历DM，计算对应DM下交互检验中各个模型的覆盖率以及对应的准确率-tr
tr_y_pro = y_pro[TrOrTe=="tr"]   # tr_y_pro  为训练集预测概率
tr_y_pred = y_pred[TrOrTe=="tr"] # tr_y_pred 为训练集预测值
tr_y_true = y_true[TrOrTe=="tr"] # tr_y_true 为训练集真实值

DM = calDM(tr_y_pro)
# Tecover = pd.DataFrame(columns=tr_y_pro.columns) # 原来
columns = [f'{col}_acc' for col in tr_y_pro.columns] + [f'{col}_mcc' for col in tr_y_pro.columns] # 新加
Tecover = pd.DataFrame(columns=columns) # 新加
cover = []
for threshold in DM:
    tr_true_DM = tr_y_true[DM<=threshold]
    tr_pred_DM = tr_y_pred[DM<=threshold]
    cover.append(len(tr_pred_DM)/len(tr_y_pro))#计算覆盖率
    
    accuracy = []
    mcc = []
    for i in range(16):        
        tr_pred_DMi = tr_pred_DM.iloc[:,i]
        accuracy.append(accuracy_score(tr_true_DM,tr_pred_DMi))
        mcc.append(matthews_corrcoef(tr_true_DM, tr_pred_DMi)) # 新加
    row_data = accuracy + mcc
    Tecover = pd.concat([Tecover, pd.DataFrame([row_data], columns=columns)], ignore_index=True)
Tecover.loc[:,"cover"] = cover
Tecover.loc[:,"threshold"] = DM

# Tecover.loc[:,"source"] = source

In [ ]:
#遍历DM，计算对应DM下交互检验中各个模型的覆盖率以及对应的准确率-te
te_y_pro = y_pro[TrOrTe=="te"]   # te_y_pro  为测试集预测概率
te_y_pred = y_pred[TrOrTe=="te"] # te_y_pred 为测试集预测值
te_y_true = y_true[TrOrTe=="te"] # te_y_true 为测试集真实值

DM = calDM(te_y_pro)
# Tecover = pd.DataFrame(columns=te_y_pro.columns) # 原来
columns = [f'{col}_acc' for col in te_y_pro.columns] + [f'{col}_mcc' for col in te_y_pro.columns] # 新加
Tecover = pd.DataFrame(columns=columns) # 新加
cover = []
for threshold in DM:
    te_true_DM = te_y_true[DM<=threshold]
    te_pred_DM = te_y_pred[DM<=threshold]
    cover.append(len(te_pred_DM)/len(te_y_pro))#计算覆盖率
    
    accuracy = []
    mcc = []
    for i in range(16):        
        te_pred_DMi = te_pred_DM.iloc[:,i]
        accuracy.append(accuracy_score(te_true_DM,te_pred_DMi))
        mcc.append(matthews_corrcoef(te_true_DM, te_pred_DMi)) # 新加
    row_data = accuracy + mcc
    Tecover = pd.concat([Tecover, pd.DataFrame([row_data], columns=columns)], ignore_index=True)
Tecover.loc[:,"cover"] = cover
Tecover.loc[:,"threshold"] = DM

# Tecover.loc[:,"source"] = source

In [ ]:
print(Tecover.columns)

In [ ]:
plotdata = Tecover.sort_values("threshold")
plotdata[plotdata.loc[:,'Model 1C_acc':'Model 8D_mcc'].std(axis=1)==0]
# plotdata.to_csv(r"Q:\ALK_1_Wild\8.ALK_300nM_DM\all_16_random_threshold_tr.csv",index=True)
plotdata.to_csv(r"Q:\ALK_1_Wild\8.ALK_300nM_DM\all_16_random_threshold_te.csv",index=True)

In [ ]:
#画累积准确率图

# label = [
#     'Model 1A','Model 1B','Model 2A','Model 2B',
#     'Model 3A','Model 3B','Model 4A','Model 4B',
#     'Model 5A','Model 5B','Model 6A','Model 6B',
#     'Model 7A','Model 7B','Model 8A','Model 8B']
label = [
    'Model 1C','Model 1D','Model 2C','Model 2D',
    'Model 3C','Model 3D','Model 4C','Model 4D',
    'Model 5C','Model 5D','Model 6C','Model 6D',
    'Model 7C','Model 7D','Model 8C','Model 8D']

colors = [
    'purple', 'green', 'blue', 'red', 
    'orange', 'pink', 'brown', 'yellow', 
    'cyan', 'magenta', 'gray', 'lime', 
    'indigo', 'coral', 'navy', 'olive']

linestyles = [
    '-', '--', ':', '-.',
    '-', '--', ':', '-.',
    '-', '--', ':', '-.',
    '-', '--', ':', '-.']

plotdata = Tecover.sort_values("threshold")

import matplotlib.pyplot as plt
fig = plt.figure(facecolor="white",figsize=(9,9))#背景改为白色，创建空画布
ax  = fig.add_subplot(111)#为画布添加绘图区域
plt.rcParams['font.family'] = ['Times New Roman']
plt.rcParams['font.size'] = 12

# 过滤数据，排除覆盖率小于20%的点
mask = plotdata.cover >= 0.2
filtered_data = plotdata[mask]
x = filtered_data.cover
for i in range(16):
    y = filtered_data.iloc[:, i]
    ax.plot(x,y,linewidth=0.4,color=colors[i],label=label[i],linestyle=linestyles[i])

ax.set_xlabel("Coverage (percent of compounds)")
ax.set_ylabel("Accuracy")
plt.title("Cumulative accuracy−coverage plot for the DM")
ax.text(0.05, 0.905, r'accuracy = 0.90',fontsize=10)
ax.text(0.05, 0.955, r'accuracy = 0.95',fontsize=10)
ax.hlines(y=0.95,xmin=-0.02, xmax=1,linewidth=0.8,linestyles="--",color='black')
ax.hlines(y=0.9,xmin=-0.02, xmax=1,linewidth=0.8,linestyles="--",color='black')

ax.legend(ncol=4,fontsize=11, loc='lower left') #(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0)
ax.axis((0,1.0,0.8,1.02)) 

def to_percent(y, position):
    return str(int(100*y)) + '%'
formatter = FuncFormatter(to_percent)
plt.gca().xaxis.set_major_formatter(formatter)
plt.savefig(r"Q:\ALK_1_Wild\8.ALK_300nM_DM\all_16_random_threshold_te.tif", dpi=600,bbox_inches="tight")
plt.show()

In [ ]:
# 打印图例-未用单元
methodList = ["SVC","DT","RF","XGBoost"]
fpList = ["MACCS","ECFP4"]
splitList = ["SOM","Random"]
label = []
for i in methodList:
    for j in fpList:
        for k in splitList:
            label.append(i+"-"+j+"-"+k)
print(label)

tr_y_true = y_true[TrOrTe=="tr"]
te_y_true = y_true[TrOrTe=="te"]
s = ""
for i in range(16):
    tr_y_pro = y_pro.loc[TrOrTe=="tr",label[i]]
    te_y_pro = y_pro.loc[TrOrTe=="te",label[i]] 
    print(label[i])

    tr_auc_value = roc_auc_score(tr_y_true, tr_y_pro)
    te_auc_value = roc_auc_score(te_y_true, te_y_pro)
    # s += f"({chr(i+97)}) The ROC curve for model {label[i]}. The AUC values of the training set and the test set are {round(tr_auc_value,3)} and {round(te_auc_value,3)}, respectively; "
    # s += f"({chr(i+97)}) The ROC curve of model {label[i]}. "
    s += f"({chr(i+97)}) {label[i]}\n"
print(s)